In [1]:
from pyspark import *
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.feature import Bucketizer

# Exploratory data analysis (EDA)

In [3]:
# Carrega dataset do desafio (basta descompactar todos na mesma pasta)
#display(dbutils.fs.ls("dbfs:/FileStore/tables/"))
df_desafio = spark.read.format("csv").options(header='true').load("/FileStore/tables/*.csv")

In [4]:
# Elimina na's e duplicados do df
df_desafio_v2 = df_desafio.dropna(how='any').dropDuplicates()

# Ajusta tipo de colunas
df_desafio_v2 = df_desafio_v2.selectExpr(
  'cast(time as timestamp) time',
  'ca',
  'unit',
  'scp',
  'station',
  'linename',
  'division',
  'desc',
  'cast(entries as int) entries',
  'cast(exits as int) exits'
)

# Features para visão temporal
df_desafio_v2 = df_desafio_v2.withColumn(
  "dt_year",
  year(col("time"))
).withColumn(
  "dt_month",
  month(col("time"))
).withColumn(
  "dt_day",
  dayofmonth(col("time"))
).withColumn(
  "dt_dayofy",
  dayofyear(col("time"))  
).withColumn(
  "dt_hour",
  hour(col("time"))
).withColumn(
  "dt_min",
  minute(col("time"))
).withColumn(
  "dt_week_no",
  weekofyear(col("time"))
).withColumn(
  "dt_int",
  unix_timestamp(col("time"))
).withColumn(
  "dt_month_year",
  date_format(col("time"), "Y-MM")
)

In [5]:
# Intervalos para buckets
splits = [-30000000, -20000000, -10000000, 0.0, 10000000, 20000000, 30000000]

# ===> Bucket: 'entries'
# dataFrame = df_desafio_v2.select(col('entries'))
# bucketizer = Bucketizer(splits=splits, inputCol="entries", outputCol="bucketedFeatures")
# bucketedData_entries = bucketizer.transform(dataFrame)
# sorted(bucketedData_entries.groupBy("bucketedFeatures").count().collect())
# [Row(bucketedFeatures=0.0, count=15721),
#  Row(bucketedFeatures=1.0, count=4028),
#  Row(bucketedFeatures=2.0, count=496),
#  Row(bucketedFeatures=4.0, count=71457425), ==> Begin
#  Row(bucketedFeatures=5.0, count=5052549),  <== End
#  Row(bucketedFeatures=6.0, count=278162),
#  Row(bucketedFeatures=7.0, count=2237294)]

# ===> Bucket: 'exits'
# dataFrame = df_desafio_v2.select(col('exits'))
# bucketizer = Bucketizer(splits=splits, inputCol="exits", outputCol="bucketedFeatures")
# bucketedData_exits = bucketizer.transform(dataFrame)
# sorted(bucketedData_exits.groupBy("bucketedFeatures").count().collect())
# [Row(bucketedFeatures=0.0, count=30576),
#  Row(bucketedFeatures=2.0, count=576),
#  Row(bucketedFeatures=3.0, count=1),
#  Row(bucketedFeatures=4.0, count=73751241), ==> Begin
#  Row(bucketedFeatures=5.0, count=3698911),  <== End
#  Row(bucketedFeatures=6.0, count=413740),
#  Row(bucketedFeatures=7.0, count=1150630)]

# Parâmetros para filtros de outliers
outlier_begin = 0
outlier_end = 20000000

df_desafio_v2 = df_desafio_v2.where((col('entries')>=outlier_begin) & (col('entries')<=outlier_end) & (col('exits')>=outlier_begin) & (col('exits')<=outlier_end))

In [6]:
# Check: 79.609.191 / 79.130.015 / 79.045.675 / 75.923.980
count_desafio = df_desafio.count()
count_desafio_na = df_desafio.dropna(how='any').count()
count_desafio_final = df_desafio.dropna(how='any').dropDuplicates().count()
count_desafio_outliers = df_desafio_v2.count()

df_amostras = sc.parallelize([
  ('antes',count_desafio,0,0,0,0),
  ('depois',0,count_desafio_final,count_desafio-count_desafio_na,count_desafio_na-count_desafio_final,count_desafio_final-count_desafio_outliers)
]).toDF(['AMOSTRAS','TOTAL','UNICO','NA','DUPLICADO','OUTLIERS'])

display(df_amostras)

In [7]:
df_amostras_v2 = sc.parallelize([
  ('',count_desafio-count_desafio_na,count_desafio_na-count_desafio_final,count_desafio_final-count_desafio_outliers)
]).toDF(['AMOSTRAS','NA','DUPLICADO','OUTLIERS'])

display(df_amostras_v2)

In [8]:
# Método para variáveis categóricas (dummys) ~14.59 minutes
lista_idx = ['ca', 'unit', 'scp', 'station', 'linename', 'division', 'desc']
indexers = [StringIndexer(inputCol=column,outputCol=column+"_idx").fit(df_desafio_v2) for column in lista_idx]
pipeline = Pipeline(stages=indexers)
df_desafio_v2 = pipeline.fit(df_desafio_v2).transform(df_desafio_v2)
#display(df_desafio_v2)

In [9]:
#df_graficos = df_desafio_v2.filter(col('dt_year')=='2017')

In [10]:
#display(df_graficos)

time,ca,unit,scp,station,linename,division,desc,entries,exits,dt_year,dt_month,dt_day,dt_dayofy,dt_hour,dt_min,dt_week_no,dt_int,dt_month_year
2017-02-26T09:00:00.000+0000,N539A,R288,00-03-02,7 AV,FG,IND,REGULAR,309016,39580,2017,2,26,57,9,0,8,1488099600,2017-02
2017-02-26T09:00:00.000+0000,R205A,R014,04-05-01,FULTON ST,2345ACJZ,IRT,REGULAR,5,139,2017,2,26,57,9,0,8,1488099600,2017-02
2017-02-26T09:00:00.000+0000,R206,R014,02-05-00,FULTON ST,2345ACJZ,IRT,REGULAR,117440632,0,2017,2,26,57,9,0,8,1488099600,2017-02
2017-02-26T09:00:00.000+0000,R247,R178,01-03-02,77 ST,6,IRT,REGULAR,1324294,12128040,2017,2,26,57,9,0,8,1488099600,2017-02
2017-02-26T09:00:00.000+0000,R327,R361,01-06-01,PELHAM PKWY,25,IRT,REGULAR,102344064,227728,2017,2,26,57,9,0,8,1488099600,2017-02
2017-02-26T09:00:00.000+0000,R532,R328,00-05-01,METS-WILLETS PT,7,IRT,REGULAR,1128330,91797,2017,2,26,57,9,0,8,1488099600,2017-02
2017-02-26T09:00:00.000+0000,R637,R451,00-06-00,WINTHROP ST,25,IRT,REGULAR,1440194,2610753,2017,2,26,57,9,0,8,1488099600,2017-02
2017-02-26T12:00:00.000+0000,A002,R051,02-03-05,59 ST,NQR456W,BMT,REGULAR,9997087,1302680,2017,2,26,57,12,0,8,1488110400,2017-02
2017-02-26T12:00:00.000+0000,B020,R263,00-03-00,AVENUE H,BQ,BMT,REGULAR,75107,69379,2017,2,26,57,12,0,8,1488110400,2017-02
2017-02-26T12:00:00.000+0000,J034,R007,00-00-02,104 ST,JZ,BMT,REGULAR,2026029440,622131328,2017,2,26,57,12,0,8,1488110400,2017-02


In [11]:
#display(df_graficos)

In [12]:
#display(df_graficos)

time,ca,unit,scp,station,linename,division,desc,entries,exits,dt_year,dt_month,dt_day,dt_dayofy,dt_hour,dt_min,dt_week_no,dt_int,dt_month_year
2017-02-26T09:00:00.000+0000,N539A,R288,00-03-02,7 AV,FG,IND,REGULAR,309016,39580,2017,2,26,57,9,0,8,1488099600,2017-02
2017-02-26T09:00:00.000+0000,R205A,R014,04-05-01,FULTON ST,2345ACJZ,IRT,REGULAR,5,139,2017,2,26,57,9,0,8,1488099600,2017-02
2017-02-26T09:00:00.000+0000,R206,R014,02-05-00,FULTON ST,2345ACJZ,IRT,REGULAR,117440632,0,2017,2,26,57,9,0,8,1488099600,2017-02
2017-02-26T09:00:00.000+0000,R247,R178,01-03-02,77 ST,6,IRT,REGULAR,1324294,12128040,2017,2,26,57,9,0,8,1488099600,2017-02
2017-02-26T09:00:00.000+0000,R327,R361,01-06-01,PELHAM PKWY,25,IRT,REGULAR,102344064,227728,2017,2,26,57,9,0,8,1488099600,2017-02
2017-02-26T09:00:00.000+0000,R532,R328,00-05-01,METS-WILLETS PT,7,IRT,REGULAR,1128330,91797,2017,2,26,57,9,0,8,1488099600,2017-02
2017-02-26T09:00:00.000+0000,R637,R451,00-06-00,WINTHROP ST,25,IRT,REGULAR,1440194,2610753,2017,2,26,57,9,0,8,1488099600,2017-02
2017-02-26T12:00:00.000+0000,A002,R051,02-03-05,59 ST,NQR456W,BMT,REGULAR,9997087,1302680,2017,2,26,57,12,0,8,1488110400,2017-02
2017-02-26T12:00:00.000+0000,B020,R263,00-03-00,AVENUE H,BQ,BMT,REGULAR,75107,69379,2017,2,26,57,12,0,8,1488110400,2017-02
2017-02-26T12:00:00.000+0000,J034,R007,00-00-02,104 ST,JZ,BMT,REGULAR,2026029440,622131328,2017,2,26,57,12,0,8,1488110400,2017-02


In [13]:
#display(df_graficos)

time,ca,unit,scp,station,linename,division,desc,entries,exits,dt_year,dt_month,dt_day,dt_dayofy,dt_hour,dt_min,dt_week_no,dt_int,dt_month_year
2017-01-04T12:54:34.000+0000,R310,R053,01-00-04,3 AV-149 ST,25,IRT,REGULAR,1957817,450079,2017,1,4,4,12,54,1,1483534474,2017-01
2017-01-01T00:00:00.000+0000,C009,R057,03-03-00,ATL AV-BARCLAY,BDNQR2345,BMT,REGULAR,2557106,2571653,2017,1,1,1,0,0,52,1483228800,2017-01
2017-01-01T00:00:00.000+0000,C012,R258,01-03-01,4AV-9 ST,DFGMNR,BMT,REGULAR,1719956,573785,2017,1,1,1,0,0,52,1483228800,2017-01
2017-01-01T00:00:00.000+0000,H027,R137,01-06-02,MYRTLE-WYCKOFF,LM,BMT,REGULAR,4046904,1646200,2017,1,1,1,0,0,52,1483228800,2017-01
2017-01-01T00:00:00.000+0000,N063,R011,02-06-00,42 ST-PORT AUTH,ACENQRS1237W,IND,REGULAR,3424061,822151,2017,1,1,1,0,0,52,1483228800,2017-01
2017-01-01T00:00:00.000+0000,N307,R359,00-00-02,COURT SQ-23 ST,EMG,IND,REGULAR,3966039,870454,2017,1,1,1,0,0,52,1483228800,2017-01
2017-01-01T00:00:00.000+0000,N504,R021,02-00-02,42 ST-BRYANT PK,BDFM7,IND,REGULAR,7760538,11118924,2017,1,1,1,0,0,52,1483228800,2017-01
2017-01-01T00:00:00.000+0000,N504,R021,02-00-05,42 ST-BRYANT PK,BDFM7,IND,REGULAR,1600395,1425570,2017,1,1,1,0,0,52,1483228800,2017-01
2017-01-01T00:00:00.000+0000,N606,R025,00-00-06,JAMAICA CENTER,EJZ,IND,REGULAR,6792606,1640736,2017,1,1,1,0,0,52,1483228800,2017-01
2017-01-01T00:00:00.000+0000,R626,R062,00-03-01,CROWN HTS-UTICA,34,IRT,REGULAR,2948677,1680128,2017,1,1,1,0,0,52,1483228800,2017-01
